Main practical assignment
=================


Genetic Algorithms for solving the Multi-size knapsack packing problem
This practical assignment requires to develop, using Python, an implementation of genetic algorithms for solving a variant of the Knapsack problem, which can be stated as follows:

Multi-size knapsack packing problem. Given a list of items L, where each item has a weight associated with it, the problem is to find a partition of the items into several subsets associated with multiple knapsacks, in such a way that the free space in the knapsacks is minimized. 
We will assume that we can use a finite number of sizes for the knapsacks (the list of allowed sizes/capacities should be provided as input).
We will assume that we can use an arbitrary number of knapsacks of the same size.

In [32]:
import random

In [33]:
# _______________________________________________________
# Knapsack problem 1:
# 10 objects, maximum weight 165
weights1 = [23,31,29,44,53,38,63,85,89,82]
values1 = [92,57,49,68,60,43,67,84,87,72]

# Optimal solution= [1,1,1,1,0,1,0,0,0,0], value= 309
# _______________________________________________________



# _______________________________________________________
# Knapsack problem 2:
# 15 objects, maximum weight 750

weights2 = [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120]
values2 = [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240]

# Optimal solution= [1,0,1,0,1,0,1,1,1,0,0,0,0,1,1], value= 1458
# _______________________________________________________



# _______________________________________________________
# Knapsack problem 3:
# 24 objects, maximum weight 6404180
weights3 = [382745,799601,909247,729069,467902, 44328,
       34610,698150,823460,903959,853665,551830,610856,
       670702,488960,951111,323046,446298,931161, 31385,496951,264724,224916,169684]
values3 = [825594,1677009,1676628,1523970, 943972,  97426,
       69666,1296457,1679693,1902996,
       1844992,1049289,1252836,1319836, 953277,2067538, 675367,
       853655,1826027, 65731, 901489, 577243, 466257, 369261]

# Optimal solution= [1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1], value= 13549094


Knapsack problem
-----------

In [34]:
class Problem_Genetic(object):
    """ Class that will be used to represent problems to be addressed via a
    generic genetic algorithm, with the following attributes:
    - genes: list of possible genes on a chromosome
    - individuals_length: length of the chromosomes
    - decode: method that receives a genotype (chromosome) and returns its
      phenotype (chromosome "interpreted" in terms of the original problem) 
    - fitness: method that assigns a score to chromosomes (acts over
      genotypes)
    - mutation: function that implements a mutation over a chromosome
    - crossover: function that implements a crossover on two chromosomes"""

    def __init__(self,genes,individuals_length,decode,fitness):
        self.genes= genes
        self.individuals_length= individuals_length
        self.decode= decode
        self.fitness= fitness

    def mutation(self, c, prob):
        cm=list(c) # makes a COPY of c
        for i in range(len(cm)):
            if random.random() < prob :
                cm[i] = random.choice(self.genes)
        return cm

    def crossover(self,c1,c2):
        pos=random.randrange(1,self.individuals_length-1)
        cr1= c1[:pos] + c2[pos:] 
        cr2= c2[:pos] + c1[pos:] 
        return [cr1,cr2]

In [35]:
def binary_to_decimal(x):
    return sum(b*(2**i) for (i,b) in enumerate(x)) 

def sq_fitness(cr, weights, max_weight):
    res = 0
    
    for i in range(len(cr)):
        res += cr[i] * weights[i]
        
    if res <= max_weight:
        return max_weight - res
    else:
        return res * 8000
    
def sq_fitness1(cr):
    return sq_fitness(cr, weights1, 165)

def sq_fitness2(cr):
    return sq_fitness(cr, weights2, 750)

def sq_fitness3(cr):
    return sq_fitness(cr, weights3, 6404180)

In [36]:
knapsack1 = Problem_Genetic([0,1], 10, binary_to_decimal, sq_fitness1)
knapsack2 = Problem_Genetic([0,1], 15, binary_to_decimal, sq_fitness2)
knapsack3 = Problem_Genetic([0,1], 24, binary_to_decimal, sq_fitness3)

In [37]:
def initial_population(pg,size):
    return [[random.choice(pg.genes) for _ in range(pg.individuals_length)] 
             for _ in range(size)]

In [38]:
def crossover_parents(pg,parents):
    pop_output = []
    for i in range(0,len(parents),2):
        pop_output += pg.crossover(parents[i],parents[i+1])
    return pop_output

In [39]:
def mutate_individuals(pg, popul, prob):
    return [pg.mutation(p,prob) for p in popul]


In [40]:
def choose(pg,popul,k,opt):
    return opt(random.sample(popul,k),key=pg.fitness)

def tournament_selection(pg,popul,n,k,opt):
    return [choose(pg,popul,k,opt) for _ in range(n)]

In [41]:
def new_generation_t(pg,k,opt,popul, n_parents,n_direct,prob_mutate):
    p1 = tournament_selection(pg,popul,n_direct,k,opt)
    p2 = tournament_selection(pg,popul,n_parents,k,opt)
    p3 = crossover_parents(pg,p2)
    p4 = p1 + p3
    pt_plus_1 = mutate_individuals(pg,p4,prob_mutate)
    return pt_plus_1

In [42]:
def genetic_algorithm_t(pg,k,opt,ngen,size,ratio_cross,prob_mutate):
    """
    problem_genetic: an instance of the class Problem_Genetic, with the optimization problem that we want to solve.
    k: number of participants on the selection tournaments.
    opt: max or min, indicating if it is a maximization or a minimization problem.
    ngen: number of generations (halting condition)
    size: number of individuals for each generation
    ratio_cross: portion of the population which will be obtained by means of crossovers.
    prob_mutate: probability that a gene mutation will take place.
    """

    p0 = initial_population(pg,size)
    threshold = round(size * ratio_cross)
    if threshold % 2 == 0:
        n_parents = threshold
    else:
        n_parents = threshold - 1
    n_direct = size - n_parents

    for _ in range(ngen):
        p0 = new_generation_t(pg,k,opt,p0, n_parents,n_direct,prob_mutate)

    res = opt(p0, key = pg.fitness)
    return str(pg.decode(res)) + ',' + str(pg.fitness(res))

Multi-Knapsack problem
-----------

In [43]:
def range_decode(x, size):
    res = {}

    for i in range(size + 1):
        chromosomes = []
        for (j,b) in enumerate(x):
            if x[j] == i:
                chromosomes.append(1)
            else:
                chromosomes.append(0)
        res[i] = sum(b*(2**i) for (i,b) in enumerate(chromosomes))

    return res

In [44]:
def decimal_to_binary(x):
    res = []

    binary = '{0:b}'.format(int(x))
    for i in range(len(binary)):
        res.append(int(binary[i]))

    return res

In [45]:
def sq_fitness_multi(cr, weights, max_weight):
    res = 0
    
    for i in range(len(cr)):
        res += cr[i] * weights[i]
        
    if res <= max_weight:
        return max_weight - res
    else:
        return res * 8000

def sq_knapsack_fitness(cr, weights, max_weights):
    res = 0
    
    for i in range(len(max_weights)):
        res += sq_fitness_multi(decimal_to_binary(cr[i + 1]), weights, max_weights[i])
        
    return res

In [46]:
max_weights1 = [150, 100, 82]
def sq_knapsack_fitness1(cr):
    return sq_knapsack_fitness(cr, weights1, max_weights1)

max_weights2 = [300, 200, 164]
def sq_knapsack_fitness2(cr):
    return sq_knapsack_fitness(cr, weights2, max_weights2)

max_weights3 = [1200000, 780000, 3500000]
def sq_knapsack_fitness3(cr):
    return sq_knapsack_fitness(cr, weights3, max_weights3)


In [47]:
multiKnapsack1 = Problem_Genetic([0,1,2,3], 10, range_decode, sq_knapsack_fitness1)
multiKnapsack2 = Problem_Genetic([0,1,2,3], 15, range_decode, sq_knapsack_fitness2)
multiKnapsack3 = Problem_Genetic([0,1,2,3], 24, range_decode, sq_knapsack_fitness3)

In [48]:
def genetic_algorithm_t_multi(pg,k,opt,ngen,size,ratio_cross,prob_mutate,knapsack_size):
    """
    problem_genetic: an instance of the class Problem_Genetic, with the optimization problem that we want to solve.
    k: number of participants on the selection tournaments.
    opt: max or min, indicating if it is a maximization or a minimization problem.
    ngen: number of generations (halting condition).
    size: number of individuals for each generation.
    ratio_cross: portion of the population which will be obtained by means of crossovers.
    prob_mutate: probability that a gene mutation will take place.
    knapsack_size: total number of knapsacks to consider.
    """

    p0 = initial_population(pg,size)
    threshold = round(size * ratio_cross)
    if threshold % 2 == 0:
        n_parents = threshold
    else:
        n_parents = threshold - 1
    n_direct = size - n_parents

    for _ in range(ngen):
        p0 = new_generation_t(pg,k,opt,p0, n_parents,n_direct,prob_mutate)

    res = opt(p0, key = pg.fitness)
    return str(pg.decode(res,knapsack_size)) + ', ' + str(pg.fitness(res))

Data generation
-----------

In [49]:
import csv

instances = 300

In [50]:
with open('knapsack1.csv', 'w', newline='') as csvfile:
    reader = csv.writer(csvfile)
    reader.writerow(['chromosome','fitness'])
    for _ in range(instances):
        row = genetic_algorithm_t(knapsack1,3,min,300,8,0.7,0.3).split(',')
        reader.writerow(row)

In [ ]:
with open('knapsack2.csv', 'w', newline='') as csvfile:
    reader = csv.writer(csvfile)
    reader.writerow(['chromosome','fitness'])
    for _ in range(instances):
        row = genetic_algorithm_t(knapsack2,3,min,300,8,0.7,0.3).split(',')
        reader.writerow(row)

In [ ]:
instances_knapsack3 = 5000
with open('knapsack3.csv', 'w', newline='') as csvfile:
    reader = csv.writer(csvfile)
    reader.writerow(['chromosome','fitness'])
    for _ in range(instances_knapsack3):
        row = genetic_algorithm_t(knapsack3,3,min,300,8,0.7,0.3).split(',')
        reader.writerow(row)

In [ ]:
with open('multiKnapsack1.csv', 'w', newline='') as csvfile:
    reader = csv.writer(csvfile)
    reader.writerow(['remaining_obj','ac_weight','knapsack','ac_weight','knapsack','ac_weight','knapsack','ac_weight','total_fitness'])
    for _ in range(instances):
        row = genetic_algorithm_t_multi(multiKnapsack1,3,min,300,8,0.7,0.3,len(max_weights1)).replace('{','').replace('}','').replace(':',',').split(',')
        reader.writerow(row)

In [ ]:
with open('multiKnapsack2.csv', 'w', newline='') as csvfile:
    reader = csv.writer(csvfile)
    reader.writerow(['remaining_obj','ac_weight','knapsack','ac_weight','knapsack','ac_weight','knapsack','ac_weight','total_fitness'])
    for _ in range(instances):
        row = genetic_algorithm_t_multi(multiKnapsack2,3,min,300,8,0.7,0.3,len(max_weights2)).replace('{','').replace('}','').replace(':',',').split(',')
        reader.writerow(row)

In [ ]:
with open('multiKnapsack3.csv', 'w', newline='') as csvfile:
    reader = csv.writer(csvfile)
    reader.writerow(['remaining_obj','ac_weight','knapsack','ac_weight','knapsack','ac_weight','knapsack','ac_weight','total_fitness'])
    for _ in range(instances):
        row = genetic_algorithm_t_multi(multiKnapsack3,3,min,300,8,0.7,0.3,len(max_weights3)).replace('{','').replace('}','').replace(':',',').split(',')
        reader.writerow(row)